## We are using **RNN LSTM** to keep the sequence information as we need to keep the information about the complex sentences like I love this movie but not as much that movie. In order to process the complex reviews, we need to maintain the sequence. LSTMs are also known as memory networks. 

In [2]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

# fix random seed for reproducibility
numpy.random.seed(1234)

Using TensorFlow backend.


In [3]:
#load the dataset but onl keep top 5000 words due to processing problem
top_words = 5000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=top_words)

17465344/17464789 [==============================] - 0s 0us/step


In [4]:
x_train[4]

[1,
 249,
 1323,
 7,
 61,
 113,
 10,
 10,
 13,
 1637,
 14,
 20,
 56,
 33,
 2401,
 18,
 457,
 88,
 13,
 2626,
 1400,
 45,
 3171,
 13,
 70,
 79,
 49,
 706,
 919,
 13,
 16,
 355,
 340,
 355,
 1696,
 96,
 143,
 4,
 22,
 32,
 289,
 7,
 61,
 369,
 71,
 2359,
 5,
 13,
 16,
 131,
 2073,
 249,
 114,
 249,
 229,
 249,
 20,
 13,
 28,
 126,
 110,
 13,
 473,
 8,
 569,
 61,
 419,
 56,
 429,
 6,
 1513,
 18,
 35,
 534,
 95,
 474,
 570,
 5,
 25,
 124,
 138,
 88,
 12,
 421,
 1543,
 52,
 725,
 2,
 61,
 419,
 11,
 13,
 1571,
 15,
 1543,
 20,
 11,
 4,
 2,
 5,
 296,
 12,
 3524,
 5,
 15,
 421,
 128,
 74,
 233,
 334,
 207,
 126,
 224,
 12,
 562,
 298,
 2167,
 1272,
 7,
 2601,
 5,
 516,
 988,
 43,
 8,
 79,
 120,
 15,
 595,
 13,
 784,
 25,
 3171,
 18,
 165,
 170,
 143,
 19,
 14,
 5,
 2,
 6,
 226,
 251,
 7,
 61,
 113]

In [5]:
#max word sequence that comes up is 4999 since we allow only first 5000 wods to load
max_number = 0
for x in x_train:
  for y in x:
    if y > max_number:
      max_number = y

print(max_number)

4999


In [0]:
# truncate and pad the input sequences
#pading is always done in front
max_review_length = 500
x_train = sequence.pad_sequences(x_train, maxlen=max_review_length)
x_test = sequence.pad_sequences(x_test, maxlen=max_review_length)

In [7]:
x_train[4]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [37]:
#create the model
embedding_vector_length = 50
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=64)

<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x7fbb32441cc0>>
Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 380s 15ms/step - loss: 0.4349 - acc: 0.7920 - val_loss: 0.3619 - val_acc: 0.8473
Epoch 2/3
25000/25000 [==============================] - 378s 15ms/step - loss: 0.2713 - acc: 0.8943 - val_loss: 0.3015 - val_acc: 0.8724
Epoch 3/3
25000/25000 [==============================] - 373s 15ms/step - loss: 0.2264 - acc: 0.9123 - val_loss: 0.3074 - val_acc: 0.8733


In [38]:
# Evaluate the model
scores = model.evaluate(x_test, y_test, verbose= 0)
print(scores[1]*100)

87.332


## Now, the above model was very naive model. Let us add some regularization to the model. For now, I will add **dropout layer** in order to redice the over fitting aspects.

In [39]:
from keras.layers import Dropout

model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
model.add(Dropout(0.15))
model.add(LSTM(100))
model.add(Dropout(0.153))
model.add(Dense(1, activation="softmax"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, epochs=3, batch_size=64)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 50)           250000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500, 50)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
________________________________________________________________

In [40]:
# Evaluate the model
scores = model.evaluate(x_test, y_test, verbose= 0)
print(scores[1]*100)

50.0


In [41]:
from keras.layers import Dropout

model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
model.add(Dropout(0.15))
model.add(LSTM(100))
model.add(Dropout(0.153))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, epochs=3, batch_size=64)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 50)           250000    
_________________________________________________________________
dropout_3 (Dropout)          (None, 500, 50)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
25000/25000 [==============================] - 327s 13ms/step - loss: 0.4648 - acc: 0.7751
Epoch 2/

In [42]:
# Evaluate the model
scores = model.evaluate(x_test, y_test, verbose= 0)
print(scores[1]*100)

87.38


In [0]:
# Evaluate the model
scores = model.evaluate(x_test, y_test, verbose= 0)
print(scores[1]*100)

## Keras provides the capability with parameters on the LSTM layer and the dropout layer for configuring the **input dropout and recurrent_dropout** for configuring the recurrent dropout.

In [49]:
# create model with double dropout i.e. dropout between layers and 
# dropout within layers of LSTM

model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length = max_review_length))
model.add(LSTM(100, dropout=0.153, recurrent_dropout=0.153))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, epochs=3, batch_size=64)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 500, 50)           250000    
_________________________________________________________________
lstm_11 (LSTM)               (None, 100)               60400     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 101       
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
25000/25000 [==============================] - 362s 14ms/step - loss: 0.4715 - acc: 0.7769
Epoch 2/3
25000/25000 [==============================] - 360s 14ms/step - loss: 0.3492 - acc: 0.8564
Epoch 3/3
25000/25000 [==============================] - 364s 15ms/step - loss: 0.3732 - acc: 0.8437


In [50]:
# Evaluate the model
scores = model.evaluate(x_test, y_test, verbose= 0)
print(scores[1]*100)

73.11999999999999


## Lets us evaluate the model using **LSTM and CNN**.

In [0]:
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [10]:
embedding_vector_length = 50
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length = max_review_length))
model.add(Conv1D(filters=35, kernel_size=1, padding='same', activation="relu"))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, epochs=5, batch_size=128)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 50)           250000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 35)           1785      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 250, 35)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               54400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 306,286
Trainable params: 306,286
Non-trainable params: 0
_________________________________________________________________
None



Epoch 1

In [11]:
# Evaluate the model
scores = model.evaluate(x_test, y_test, verbose= 0)
print(scores[1]*100)

87.044


#CNN + FLATTEN + DENSE


In [15]:
from keras.layers import Flatten

model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length = max_review_length))
model.add(Conv1D(filters=35, kernel_size=1, padding="same", activation="relu"))
model.add(Flatten())
model.add(Dense(250, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
print(model.summary())
model.fit(x_train, y_train, epochs=6, batch_size=64)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 50)           250000    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 500, 35)           1785      
_________________________________________________________________
flatten_3 (Flatten)          (None, 17500)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 250)               4375250   
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 251       
Total params: 4,627,286
Trainable params: 4,627,286
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/6
25000/25000 [==============================] - 47s 2ms/step - loss: 0.4006 - acc: 0.8020
Epoch 

In [18]:
# Evaluate the model
scores = model.evaluate(x_test, y_test, verbose= 2)
print(scores[1]*100)

85.844


## **Predicting the outputs**

In [46]:
model.predict_classes(x_test)

array([[0],
       [1],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int32)

In [36]:
x_test

array([0, 1, 1, ..., 0, 0, 0])

In [43]:
x_test[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   